<a href="https://colab.research.google.com/github/arnaudstdr/tweet_sentiment_analysis/blob/main/tweet_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANALYSE DES SENTIMENTS SUR TWITTER
Ce projet permet d'analyser les sentiments exprimés dans des tweets enutilisantdes techniques de traitement du langage naturel (NLP) et de deep learning.
Adapté pour un dataset Kaggle avec des fichiers d'entraînement et de validation séparés.

## Sommaire
- <a href="#importation">1. Importation des bibliothèques</a>
- <a href="#ressources-nltk">2. Téléchargment des ressources NLTK</a>
- <a href="#charg-explo-donnees">3. Chargement et Exploration des données</a>
- <a href="#pretraitement">4. Prétraitement du texte</a>
- <a href="#visualisation">5. Visualisation des données</a>
- <a href="#tradi-tdidf">6. Approche traditionnelle avec TF-IDF</a>

## 1. <a id="importation">Importation des bibliothèques</a>

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
# import gradio as gr

## 2. <a id="ressources-nltk">Téléchargement des ressources NLTK</a>
L'importation des bibliothèques seules ne suffit pas pour que `nltk` fonctionne correctement. Les ressources comme les stopwords, le tokeniseur et le lemmatiseur ne sont pas incluses par défaut. Elles doivent être téléchargées séparément.



In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## 3. <a id="charg-explo-donnees">Chargment et Exploration des données</a>

In [7]:
def load_data(train_path, val_path=None):
  # Chargement du dataset d'entraînement
  train_data = pd.read_csv(train_path)
  print(f"Diemnsion du dataset d'entraînement : {train_data.shape}")
  print("Aperçu du dataset d'entraînement : ")
  print(train_data.head())

  # Vérification des colonnes du dataset
  print("\nColonnes du dataset d'entraînement : ")
  print(train_data.columns.tolist())

  # si un fichier de validation
  if val_path:
    val_data = pd.read_csv(val_path)
    print("\nDimensions du dataset de validation : {val_data.shape}")
    print(val_data.head())
    return train_data, val_data
  else:
    return train_data, None

## 4. <a id="pretraitement">Prétraitement du texte</a>
Pourquoi le prétraitement est important ?
1. **Réduction du bruit** : Les tweets contiennent souvent beucoup d'éléments non pertinents pour l'analyse des sentiments (URLs, mentions, etc.).
2. **Normalisation** : Les différentes formes d'un même mot (pluriels, conjugaisons) sont ramenées à une forme santard.
3. **Réduction de la dimensionnalité** : En supprimant les stopwords et en utilisant la lemmatisation, on réduit le nombre de mots uniques, ce qui facilite l'apprentissage des modèles.
4. **Amélioration des performances** : Un texte bien prétraité permet aux modèles de se concentrer sur les mots et expressions qui véhiculent réeelement un sentiment.  

In [4]:
def preprocess_text(text):
  # Vérifier si le texte est une chaîne de caractères
  if not isinstance(text, str):
    return ""

  # Convertir en minuscules
  text = text.lower()

  # Supprimer les URLs
  text = re.sub(r'@\w+', '', text)

  # Supprimer les hashtags
  text = re.sub(r'#\w+', '', text)

  # Supprimer les caractères non-alphanumériques
  text = re.sub(r'[^\w\s]', '', text)

  # Supprimer les chiffres
  text = re.sub(r'\d+', '', text)

  # Tokenisation
  tokens = word_tokenize(text)

  # Supression des stopwords
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]

  # Lemmatisation
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in tokens]

  # Rejoindre les tokens
  return ' '.join(tokens)

In [5]:
"""Applique le prétraitement à l'ensemble du dataste"""
def prepare_datdset(data, text_column, label_column):
  # Vérifier que les colonnes existent
  if text_column not in data.columns:
    raise ValueError(f"Les colonnes de texte '{text_column}' n'existe pas dans le dataset")
  if label_column not in data.columns:
    raise ValueError(f"La colonne d'étiquette '{label_column}' n'existe pas dans le datatset")

  # Appliquer le prétraitement au text
  data['clean_text'] = data[label_column].apply(preprocess_text)

  # Identifier les valeurs uniques dans la colonne des sentiments
  unique_sentiments = data[label_column].unique()
  print(f"Valeurs uniques de sentiment trouvées : {unique_sentiments}")

  # Créer un mapping des sentiments basé sur les valeurs trouvées
  # Partie pouvant nécessitant un ajustement en fonction du format du dataset
  sentiment_map = {}

  # Essayer de détecter automatiquement le format du sentiment
  if set(unique_sentiments).issubset({0, 1}) or set(unique_sentiments).issubset({'0', '1'}):
    # Dataset binaire (positif/négatif)
    sentiment_map = {0: 0, 1: 1, '0': 0, '1': 1}
    print("Format détecté : Bianire (négatif/positif)")
  elif set(unique_sentiments).issubset({-1, 0, 1}) or set(unique_sentiments).issubset({'-1', '0', '1'}):
    # Dataset ternaire avec -1, 0, 1
    sentiment_map = {-1: 0, 0: 2, 1:1, '-1':0, '0':2, '1':1}
    print("Format détecté : Ternaire (-1=négatif, 0=neutre, 1=positif)")
  elif any(isinstance(x, str) and x.lower() in ['positive', 'negative', 'neutrel'] for x in unique_sentiments):
    # Dataset avec texte
    sentiment_map = {'negative': 0, 'neutral': 2, 'positive': 1}
    print("Format détecté : Textuel (negative/neutral/positice)")
  else:
    # Format non reconnu, créer un mapping générique
    sentiment_map = {val: idx for idx, val in enumerate(unique_sentiments)}
    print(f"Format non reconnu. Mapping créé : {sentiments}")

  # Appliquer le mapping
  data['sentiment_label'] = data[label_column].map(sentiment_map)

  # Vérifier qu'il n'y pas de NaN dans les labels après le mapping
  if data['sentiment_label'].isna().any():
    print("ATTENTION : Certaines valeurs de sentiment n'ont pas pu être converties !")
    print("Valeurs problématiques : ", data[data['sentiment_label'].isna()][label_column].unique())
    # Remplir les NaN avec une valeur par défaut (par exemple, 0 pour négatif)
    data['sentiment_label'] = data['sentiment_label'].fillna(0)

  return data


## 5. <a id="visualisation">Visualisation des données</a>

- La partie visualisation des données est conçue pour explorer et comprendre les caractéristiques de dataset avant de passer à la modélisation.
- La fonction `visualize_data()` crée trois visaulisations principales pour analyser la distribution et les caractéristiques des sentiments dans les tweets.

In [6]:
def visualize_data(data, sentiment_column, text_column):
  """Crée des visualisations pour explorer le dataset"""
  # Distribution des sentiments
  plt.figure(figsize=(10, 6))
  sentiment_counts = data[sentiment_column].value_counts()

  #Création du palette de couleurs plus attrayante
  colors = ['#ff9999', '#66b3ff', '#99ff99'][:len(sentiment_counts)]

  # Affichage du graphique
  ax = sentiment_counts.plot(kind='bar', color=colors)
  for i, v in enumerate(sentiment_counts):
    ax.text(i, v + 0.1, str(v), ha='center')

  plt.title('Distribution des Sentiments', fontsize=14)
  plt.xlabel('Sentiment', fontsize=12)
  plt.ylabel('Nombre de Tweets', fontsize=12)
  plt.tight_layout()
  plt.savefig('sentiment_distribution.png')
  print("Graphique de distributiondes sentiments sauvegardé dans 'sentiment_distribution.png")

  # Longueur des tweets par sentiment
  data['text_length'] = data[text_column].astype(str).apply(len)

  plt.figure(figsize=(12, 7))

  # Utilisation de box plot avec swarmplot pour une meilleure visualisation
  ax = sns.boxplot(x=sentiment_column, y='text_legnth', data=data, palette='Set2')
  sns.swarmplot(x=sentiment_column, y='text_length', data=data, color='0.25', size=4, alpha=0.5)

  plt.title('Longueyr des Tweets par Sentiment', fontsize=14)
  plt.xlabel('Senriment', fontsize=12)
  plt.ylabel('Longueur du Tweet (caractères)', fontsize=12)
  plt.tight_layout()
  plt.savefig('tweet_length_by_sentiment.png')
  print("Graphique de longueur des tweets sauvegardé dans 'tweet_by_length_sentiment.png'")

  # Analyse des mots les plus fréquents par sentiment
  from collections import Counter
  import matplotlib.cm as cm

  # Créer un DataFrame pour les mots les plus fréquents par sentiment
  plt.figure(figsize=(15, 12))

  # Définir le nombre de sentiments dans le dataset
  num_sentiments = data['sentiment_label'].nunique()

  # Ajuster le nombre de sous-graphique en fonction du nombre de sentiment
  fig, axes = plt.subplots(1, num_sentiments, figsize=(15, 6))
  if num_sentiments == 1:
    axes = [axes]     # Assure que axes est toujours une liste

  sentiment_names = {0: 'Négatif', 1: 'Positif', 2: 'Neutre'}

  # Pour chaque sentiment, trouver les mots les plus fréquents
  for i, sentiment_value in enumerate(sorted(data['sentiment_label'].unique())):
    # Filtrer les tweets par sentiment
    sentiment_data = data[data['sentiment_label'] == sentiment_value]

    # Joindre tous les textes nettoyés
    all_words = ' '.join(sentiment_data['clean_text'].astype(str)).split()

    # Compter les mots
    word_counts = Counter(all_words)

    # Prendre les 15 mots les plus fréquents
    most_common = word_counts.most_common(15)

    # Créer des listes pour les graphiques
    words = [word for word, count in most_common]
    counts = [count for word, count in most_common]

    # Tracer le graphique à barre horizontales
    sentiment_name = sentiment_names.get(sentiment_value, f"Sentiment {sentiment_value}")
    axes[i].barh(words, counts, color=cm.Set3(i / num_sentiments))
    axes[i].set_title(f'Mots fréquent - {sentiment_name}')
    axes[i].set_xlabel('Fréquence')

  plt.tight_layout()
  plt.savefig('frequent_words_by_sentiment.png')
  print("Graphique des mots fréquents sauvegardé dans 'frequent_words_by_sentiment.png'")

  return

Ces visulisations constituent une étape d'analyse exploratoire des données (EDA) importante qui aide à :
- Comprendre le déséquilibre éventuel des classes.
- Voir si la longueur des tweets est corrélée au sentiment.
- Identifier les mots caractéristiques de chaque sentiment.

## 6. <a id="tradi-tdidf">Approche Traditionnelle avec TF-IDF</a>

Cette section implémente une méthode classqieu d'analyse de sentiments basé sur des techniques de NLP plus traditionnelles, avant l'ère des transformers.

In [ ]:
def train_tfidf_model(train_data, val_data=None):
